In [5]:
#Librerias para ML

#Manejo de la base de datos
import pandas as pd
import numpy as np

#Escalar Variables
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

#Separar los datos entrenamiento y prueba
from sklearn.model_selection import train_test_split

#Evaluación del modelo
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import median_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.model_selection import cross_val_score
#from bayes_opt import BayesianOptimization
from hyperopt import hp,fmin,tpe,Trials,space_eval

#Creación de modelo
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.ensemble import StackingRegressor
from lightgbm import LGBMRegressor
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from skelm import ELMRegressor
#import keras_tuner as kt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.regularizers import l2
from keras.callbacks import EarlyStopping

import statsmodels.api as sm

#Para graficas
import matplotlib.pyplot as plt
import seaborn as sns

#Private functions
from own_functions import residuos
from own_functions import comparisson
from own_functions import scorer

# 1. Cargando las BD

In [6]:
#Se crea un dataframe con los datos tratados
df_estandar = pd.read_csv('Train_prep_iter6.csv')

In [7]:
# #Se crea un dataframe con los datos tratados
# df_minmax = pd.read_csv('Train_prep_iter6.csv')

## 1.1 División de la base de datos en datos de entrada y salida

In [8]:
# Separamos de nuestros datos la variable de salida
output_name = ['load_shortfall_3h']

In [9]:
snow_3h = [x for x in df_estandar.columns if 'snow_3h' in x]
rain_3h = [x for x in df_estandar.columns if 'rain_3h' in x]
print(snow_3h,rain_3h)

['Bilbao_snow_3h', 'Valencia_snow_3h'] ['Seville_rain_3h', 'Barcelona_rain_3h']


In [10]:
clouds  = [x for x in df_estandar.columns if 'clouds_all' in x]
clouds.pop(0)
print(clouds)

['Seville_clouds_all', 'Madrid_clouds_all']


In [11]:
to_drop_1 = []
to_drop_1.extend(snow_3h)
to_drop_1.extend(rain_3h)

In [12]:
# Separación de las variables

#BD Estandarizada
x_est = df_estandar.drop(columns=to_drop_1).drop(columns = output_name)
y_est = df_estandar[output_name]

In [13]:
columns_x1=[x for x in x_est.columns if 'Estación' not in x and 'weather_id' not in x]

In [14]:
columns_x2=[x for x in x_est.columns if 'Estación' in x or 'weather_id' in x]

In [15]:
x_est1=x_est[columns_x1]
x_est2=x_est[columns_x2]

In [16]:
scaler_x = StandardScaler()
#nombres = x_est.columns
x_est1 = scaler_x.fit_transform(x_est1)
x_est1 = pd.DataFrame(x_est1, columns = columns_x1)
x_est=pd.concat([x_est2,x_est1],axis=1)

scaler_y = StandardScaler()
nombres = y_est.columns
y_est = scaler_y.fit_transform(y_est)
y_est = pd.DataFrame(y_est, columns = nombres)

# 2. Datos de entrenamiento y prueba 

In [19]:
# División de los datos en train y test
#En este caso, el 80% de los datos se utilizarán para entrenar el modelo. 

#Para los datos estandarizados
X_train_est, X_test_est, y_train_est, y_test_est = train_test_split(
                                        x_est,
                                        y_est.values.reshape(-1,1),
                                        train_size   = 0.9,
                                        random_state = 1234,
                                        shuffle      = True
                                    )

# 3. Creación de los modelos

## 3.1 Random Forest

In [22]:
#Espacio de hiperparametros para el Random Forest
RF_space = {
    'n_estimators'      : hp.choice('n_estimators',[10, 50, 100, 300, 750, 1200,1300,1500]),
    'criterion'         : hp.choice('criterion',['squared_error', 'absolute_error', 'friedman_mse']),
    'min_samples_split' : hp.quniform('min_samples_leaf', 2, 10, 1),
    'min_samples_leaf'  : hp.quniform ('min_samples_split', 1, 10, 1),
    'max_features'      : hp.choice('max_features', [1.0, 'sqrt','log2', None]),
}

In [23]:
a=hp.choice('criterion',['squared_error', 'absolute_error', 'friedman_mse'])

In [24]:
def RF_objective(space):
    RF = RandomForestRegressor(  
        n_estimators      = space['n_estimators'],
        criterion         = space['criterion'], 
        min_samples_leaf  = int(space['min_samples_leaf']),
        min_samples_split = int(space['min_samples_split']),
        max_features      = space['max_features'],
        n_jobs            = -1,
    )
    
    MAE = cross_val_score(RF, X_train_est, y_train_est.ravel(), scoring='neg_mean_absolute_error', cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return -1*MAE

## 3.2 Support Vector Machine 

In [25]:
#Espacio de hiperparametros para La maquina de vectores de soporte
svr_space = {
    'kernel'    : hp.choice('kernel',['linear','poly','rbf','sigmoid','precomputed']),
    'degree'    : hp.choice('degree', [1, 5, 10]),
    'gamma'     : hp.choice('gamma', ['scale','auto']),
    'coef0'     : hp.choice('coef0', [3, 5, 10]),
    'C'         : hp.choice('C', [0.1, 0.5, 1.0]),
    'epsilon'   : hp.choice('epsilon', [0.1, 0.5, 1.0]),
}

In [26]:
def SVR_objective(space):
    svr = SVR(  
        kernel  = space['kernel'],
        degree  = space['degree'], 
        gamma   = space['gamma'],
        coef0   = space['coef0'],
        C       = space['C'],
        epsilon = space['epsilon'],
    )
    
    MAE = cross_val_score(svr, X_train_est, y_train_est.ravel(), scoring='neg_mean_absolute_error', cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return -1*MAE

## 3.3 Extratrees Regressor

In [27]:
EXTR_space = {
    'n_estimators'      : hp.choice('n_estimators',[10, 50, 100, 300, 750, 1200,1300,1500,]), 
    'criterion'         : hp.choice('criterion',['squared_error', 'absolute_error', 'friedman_mse']), 
    'min_samples_split' : hp.uniform('min_samples_split', 2, 10), 
    'min_samples_leaf'  : hp.uniform('min_samples_leaf', 1, 10),  
    'max_features'      : hp.choice('max_features', [1.0, 'sqrt','log2', None]),   
    'n_jobs'            : -1, 
    'random_state'      : 1234,      
}


In [28]:
def EXTR_objective(space):
    EXTR = ExtraTreesRegressor(  
        n_estimators      = space['n_estimators'],
        criterion         = space['criterion'], 
        min_samples_leaf  = int(space['min_samples_leaf']),
        min_samples_split = int(space['min_samples_split']),
        max_features      = space['max_features'],
        n_jobs            = space['n_jobs'],
        random_state      = space['random_state'],
    )
    
    MAE = cross_val_score(EXTR, X_train_est, y_train_est.ravel(), scoring='neg_mean_absolute_error', cv = 10, n_jobs=-1).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return -1*MAE

## 3.4 Light Gradient Boosting Machine

In [29]:
LGBMR_space = {
    'boosting_type'     : hp.choice('boosting_type',['gbdt','dart','rf']), 
    'num_leaves'        : hp.quniform('num_leaves', 1, 100, 10), 
    'learning_rate'     : hp.choice('learning_rate', [0.01, 0.5, 0.9]), 
    'n_estimators'      : hp.choice('n_estimators',[10, 50, 100, 300, 750, 1200,1300,1500,]), 
    'subsample_for_bin' : 200000,   
    'min_split_gain'    : hp.choice('min_split_gain', [0.0, 0.5, 1.0]), 
    'min_child_weight'  : 0.001, 
    'min_child_samples' : hp.choice('min_child_samples', [10, 20, 50, 100]),   
    'reg_alpha'         : hp.choice('reg_alpha', [0.0, 0.0001, 0.000001]), 
    'reg_lambda'        : hp.choice('reg_lambda', [0.0, 0.0001, 0.000001]), 
    'random_state'      : 1234, 
    'n_jobs'            : -1, 
    'importance_type'   : hp.choice('importance_type', ['split', 'gain']),    
}

In [30]:
def LGBMR_objective(space):
    LGBMR = LGBMRegressor(  
        boosting_type     = space['boosting_type'],
        num_leaves        = int(space['num_leaves']), 
        learning_rate     = space['learning_rate'], 
        n_estimators      = space['n_estimators'], 
        subsample_for_bin = space['subsample_for_bin'],   
        min_split_gain    = space['min_split_gain'], 
        min_child_weight  = space['min_child_weight'], 
        min_child_samples = space['min_child_samples'],  
        reg_alpha         = space['reg_alpha'], 
        reg_lambda        = space['reg_lambda'], 
        random_state      = space['random_state'], 
        n_jobs            = space['n_jobs'], 
        importance_type   = space['importance_type'],  
    )
    
    MAE = cross_val_score(LGBMR, X_train_est, y_train_est.ravel(), scoring='neg_mean_absolute_error', cv = 5).mean()

    # We aim to maximize accuracy, therefore we return it as a negative value
    return -1*MAE

## 3.5 Deep Neural Network

In [31]:
# Ver el notebook 7A


# 4. Hypertunnig

In [32]:
# Sintonización del modelo Random Forest
trials_RF = Trials()
best_RF = fmin(fn= RF_objective, space= RF_space, algo= tpe.suggest, max_evals = 2, trials= trials_RF)

In [33]:
#RF_best_params = RF_optimizer.max['params']
best_RF_parameters = space_eval(RF_space,best_RF)
RF_best = RandomForestRegressor(
    n_estimators      = best_RF_parameters['n_estimators'],
    criterion         = best_RF_parameters['criterion'], 
    min_samples_leaf  = best_RF_parameters['min_samples_leaf'],
    min_samples_split = best_RF_parameters['min_samples_split'],
    max_features      = best_RF_parameters['max_features'],
    n_jobs            = -1,)

In [34]:
RF_best.fit(X_train_est,y_train_est.ravel())

In [38]:
# Sintonización del modelo Extra Trees Regressor
trials_EXTR = Trials()
best_EXTR = fmin(fn= EXTR_objective, space= EXTR_space, algo= tpe.suggest, max_evals = 100, trials= trials_EXTR)

  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

 26%|██▌       | 26/100 [11:39:24<72:19:49, 3518.78s/trial, best loss: 0.6134063080753664] 

In [ ]:
#EXTR_best_params = EXTR_optimizer.max['params']
best_EXTR_parameters = space_eval(EXTR_space,best_EXTR)
EXTR_best = ExtraTreesRegressor(
    n_estimators      = best_EXTR_parameters['n_estimators'],
    criterion         = best_EXTR_parameters['criterion'], 
    min_samples_leaf  = int(best_EXTR_parameters['min_samples_leaf']),
    min_samples_split = int(best_EXTR_parameters['min_samples_split']),
    max_features      = best_EXTR_parameters['max_features'],
    n_jobs            = best_EXTR_parameters['n_jobs'],
    random_state      = best_EXTR_parameters['random_state'],
)

In [ ]:
EXTR_best.fit(X_train_est,y_train_est.ravel())

ExtraTreesRegressor(criterion='friedman_mse', max_features=None,
                    min_samples_leaf=5, min_samples_split=9, n_estimators=1500,
                    n_jobs=-1, random_state=1234)

In [ ]:
# Sintonización del modelo Light Gradient Boosting Machine
trials_LGBMR = Trials()
best_LGBMR = fmin(fn= LGBMR_objective, space= LGBMR_space, algo= tpe.suggest, max_evals = 2, trials= trials_LGBMR)


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.002543 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 6537                    
[LightGBM] [Info] Number of data points in the train set: 6134, number of used features: 60
[LightGBM] [Info] Start training from score -0.014026
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] No further splits with positive gain, best ga

In [ ]:
#LGBMR_best_params = LGBMR_optimizer.max['params']
best_LGBMR_parameters = space_eval(LGBMR_space,best_LGBMR)
LGBMR_best = LGBMRegressor(
    boosting_type     = best_LGBMR_parameters['boosting_type'],
    num_leaves        = int(best_LGBMR_parameters['num_leaves']), 
    learning_rate     = best_LGBMR_parameters['learning_rate'], 
    n_estimators      = best_LGBMR_parameters['n_estimators'], 
    subsample_for_bin = best_LGBMR_parameters['subsample_for_bin'],   
    min_split_gain    = best_LGBMR_parameters['min_split_gain'], 
    min_child_weight  = best_LGBMR_parameters['min_child_weight'], 
    min_child_samples = best_LGBMR_parameters['min_child_samples'], 
    reg_alpha         = best_LGBMR_parameters['reg_alpha'], 
    reg_lambda        = best_LGBMR_parameters['reg_lambda'], 
    random_state      = best_LGBMR_parameters['random_state'], 
    n_jobs            = best_LGBMR_parameters['n_jobs'], 
    importance_type   = best_LGBMR_parameters['importance_type'], 
)

In [ ]:
LGBMR_best.fit(X_train_est,y_train_est.ravel())

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.001777 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 5731
[LightGBM] [Info] Number of data points in the train set: 853, number of used features: 59
[LightGBM] [Info] Start training from score 0.088045
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, b

LGBMRegressor(learning_rate=0.01, min_child_samples=10, n_estimators=1200,
              n_jobs=-1, num_leaves=90, random_state=1234, reg_alpha=0.0001,
              reg_lambda=1e-06)